## import包

In [1]:
import sys
sys.path.append('../AutoX')

In [2]:
from autox import AutoXRecommend

/home/caihengxing/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not mo

In [4]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## 读数据

In [7]:
path = '~/AutoX/autox/autox_recommend/datasets/MovieLens_AutoX/'

In [8]:
inter_df = pd.read_csv(path + 'inter_df.csv')
item_df = pd.read_csv(path + 'item_df.csv')
test = pd.read_csv(path + 'test.csv')

In [9]:
inter_df.head()

,userId,movieId,time
0,1,296,2006-05-17 15:34:04
1,1,307,2006-05-17 12:27:08
2,1,665,2006-05-17 15:13:40
3,1,1237,2006-05-17 12:27:19
4,1,2632,2006-05-17 15:04:08


## 配置参数

In [10]:
uid = 'userId'
iid = 'movieId'
time_col = 'time'
recall_num = 100

## 准备测试集结果

In [11]:
inter_df[time_col].min(), inter_df[time_col].max()

('1995-01-09 11:46:49', '2019-11-14 23:20:55')

In [12]:
test[time_col].min(), test[time_col].max()

('2019-11-15 00:08:42', '2019-11-21 09:06:53')

In [13]:
assert(test[time_col].min() > inter_df[time_col].max())

In [14]:
positive_items_test = test.groupby([uid])[iid].apply(list)
test_users = positive_items_test.keys()
test_items = []

for i, user in tqdm(enumerate(test_users)):
    test_items.append(positive_items_test[user])
    
print("Total users in testidation:", len(test_users))

283it [00:00, 168846.09it/s]

Total users in testidation: 283


## 时间列转化

In [16]:
inter_df[time_col] = pd.to_datetime(inter_df[time_col])

## 执行AutoX

In [18]:
autoXRecommend = AutoXRecommend()

autoXRecommend.fit(inter_df = inter_df, user_df = None, item_df = item_df,
                  uid = uid, iid = iid, time_col = time_col,
                  recall_num = recall_num, 
                  mode = 'recalls', recall_method = 'popular')

In [19]:
res = autoXRecommend.transform(test_users)

100%|██████████| 283/283 [00:00<00:00, 1025.37it/s]


In [20]:
res

,userId,prediction
0,4,"[318, 5952, 7153, 4993, 858, 260, 1196, 79132,..."
1,606,"[318, 5952, 7153, 4993, 858, 260, 1196, 79132,..."
2,1746,"[318, 5952, 7153, 4993, 858, 260, 1196, 79132,..."
3,3409,"[318, 5952, 7153, 4993, 858, 260, 1196, 79132,..."
4,4037,"[318, 5952, 7153, 4993, 858, 260, 1196, 79132,..."
...,...,...
278,159074,"[318, 5952, 7153, 4993, 858, 260, 1196, 79132,..."
279,159388,"[318, 5952, 7153, 4993, 858, 260, 1196, 79132,..."
280,159523,"[318, 5952, 7153, 4993, 858, 260, 1196, 79132,..."
281,161485,"[318, 5952, 7153, 4993, 858, 260, 1196, 79132,..."


## 结果查看

In [21]:
def apk(actual, predicted, k=12):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [24]:
outputs = res['prediction']
print("mAP Score on Validation set:", mapk(test_items, outputs))

mAP Score on Validation set: 0.0473412592184677
